In [1]:
import random
import pandas as pd
import numpy as np
import os
from tqdm.auto import tqdm
import librosa

from sklearn.tree import DecisionTreeClassifier
import os
import random
import warnings
from sklearn.model_selection import train_test_split

import librosa
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from tqdm.auto import tqdm
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader

from transformers import AutoModelForAudioClassification, Wav2Vec2FeatureExtractor

warnings.filterwarnings(action='ignore')
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
import warnings
warnings.filterwarnings(action='ignore')

RuntimeError: module compiled against API version 0xe but this version of numpy is 0xd

In [3]:
train_df = pd.read_csv('./train.csv')
test_df = pd.read_csv('./test.csv')
train_df.info()

train_df.reset_index(drop=True, inplace=True)
test_df.reset_index(drop=True, inplace=True)
# valid_df.reset_index(drop=True, inplace=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5001 entries, 0 to 5000
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      5001 non-null   object
 1   path    5001 non-null   object
 2   label   5001 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 117.3+ KB


In [4]:
CFG = {
    'SR':16000,
    'N_MFCC':64, # Melspectrogram 벡터를 추출할 개수
    'SEED':42
}

In [6]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
#     torch.manual_seed(seed)
#     torch.cuda.manual_seed(seed)
#     torch.backends.cudnn.deterministic = True
#     torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED']) # Seed 고정

In [ ]:
train_df = pd.read_csv('./train.csv')
test_df = pd.read_csv('./test.csv')
train_df.info()
# train_df, valid_df = train_test_split(train_df, test_size=0.2, random_state=CFG['SEED'])

def speech_file_to_array_fn(df):
    feature = []
    for path in tqdm(df['path']):
        # path = '/content/drive/MyDrive/hi/sound01' + path[1:] 
        speech_array, _ = librosa.load(path, sr=CFG['SR'])
        feature.append(1000*speech_array**3)
    return feature

train_df.reset_index(drop=True, inplace=True)
test_df.reset_index(drop=True, inplace=True)

# valid_df.reset_index(drop=True, inplace=True)

train_x = speech_file_to_array_fn(train_df)
test_x = speech_file_to_array_fn(test_df)
# valid_x = speech_file_to_array_fn(valid_df)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5001 entries, 0 to 5000
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      5001 non-null   object
 1   path    5001 non-null   object
 2   label   5001 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 117.3+ KB


  0%|          | 0/5001 [00:00<?, ?it/s]

  0%|          | 0/1881 [00:00<?, ?it/s]

In [ ]:
import matplotlib.pyplot as plt
plt.plot(1000*librosa.load(train_df['path'][1020], sr=CFG['SR'])[0]**3)

In [8]:
MODEL_NAME = "facebook/wav2vec2-base"
processor = Wav2Vec2FeatureExtractor.from_pretrained(MODEL_NAME)

class CustomDataSet(torch.utils.data.Dataset):
    def __init__(self, x, y, processor):
        self.x = x
        self.y = y
        self.processor = processor

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        input_values = self.processor(self.x[idx], sampling_rate=CFG['SR'], return_tensors="pt", padding=True).input_values
        if self.y is not None:
            return input_values.squeeze(), self.y[idx]
        else:
            return input_values.squeeze()

def collate_fn(batch):
    x, y = zip(*batch)
    x = pad_sequence([torch.tensor(xi) for xi in x], batch_first=True)
    y = pad_sequence([torch.tensor([yi]) for yi in y], batch_first=True)  # Convert scalar targets to 1D tensors
    return x, y

def create_data_loader(dataset, batch_size, shuffle, collate_fn, num_workers=0):
    return DataLoader(dataset,
                      batch_size=batch_size,
                      shuffle=shuffle,
                      collate_fn=collate_fn,
                      num_workers=num_workers
                      )

train_dataset = CustomDataSet(train_x, train_df['label'], processor)
test_dataset = CustomDataSet(test_x, y=None, processor=processor)

In [ ]:
train_df['label'].value_counts()

# 0: angry
# 1: fear
# 2: sad
# 3: disgust
# 4: neutral
# 5: happy

In [ ]:
# features = []
# path = train_df['path'][0]
        
# y, sr = librosa.load(path, sr=CFG['SR'])
# y = list(y)
# y.extend([0 for _ in range(80000-len(y))])
# features.append(y)

# len(y)

In [53]:
def get_feature_mel(df):
    features = []
    for i in tqdm(df['path']):
        # i = '/content/drive/MyDrive/hi/sound01'+i[1:]
        data, sr = librosa.load(i, sr=CFG['SR'])
        data = 1000*data**3
        n_fft = 2048
        win_length = 2048
        hop_length = 1024
        n_mels = 64
 
        D = np.abs(librosa.stft(data, n_fft=n_fft, win_length = win_length, hop_length=hop_length))
        mel = librosa.feature.melspectrogram(S=D, sr=sr, n_mels=n_mels, hop_length=hop_length, win_length=win_length)

        m_mel = mel.mean(axis=1)
        features.append(m_mel)
    return np.array(features)

train_mel = get_feature_mel(train_df)
# valid_mel = get_feature_mel(valid_df)
test_mel = get_feature_mel(test_df)

  0%|          | 0/5001 [00:00<?, ?it/s]

  0%|          | 0/1881 [00:00<?, ?it/s]

In [ ]:
import os
import random
import warnings
from sklearn.model_selection import train_test_split

import librosa
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from tqdm.auto import tqdm
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader

from transformers import AutoModelForAudioClassification, Wav2Vec2FeatureExtractor

warnings.filterwarnings(action='ignore')

In [ ]:
import matplotlib.pyplot as plt
plt.plot(train_mel[0])

In [28]:
def get_mfcc_feature(df):
    features = []
#     for path in tqdm(df['path']):
        
#         y, sr = librosa.load(path, sr=CFG['SR'])
#         y = list(y)
#         y.extend([0 for _ in range(80100-len(y))])
#         features.append(y)
    for path in tqdm(df['path']):
        # librosa패키지를 사용하여 wav 파일 load
        y, sr = librosa.load(path, sr=CFG['SR'])
        y = 1000*y**3
        # librosa패키지를 사용하여 mfcc 추출
        mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=CFG['N_MFCC'])
        y_feature = []
        # 추출된 MFCC들의 평균을 Feature로 사용
        for e in mfcc:
            y_feature.append(np.mean(e))
        features.append(y_feature)
    return features
    # return pd.DataFrame(features,columns=['freq'])

vector = get_mfcc_feature(train_df)
test_mfcc = get_mfcc_feature(test_df)

  0%|          | 0/5001 [00:00<?, ?it/s]

  0%|          | 0/1881 [00:00<?, ?it/s]

In [ ]:
plt.plot(vector[0])

In [ ]:
# train_df[train_df['label'] == 0].index

In [ ]:
# from sklearn.decomposition import PCA
# pca 

# pd.DataFrameFrame(vector.iloc[])

In [ ]:
# from sklearn.decomposition import PCA
# pca = PCA(n_components=600)
# pca.fit(vector)
# target = pd.DataFrame(pca.transform(vector))
# target.to_csv('./origin_600_pca.csv')

In [ ]:
# import numpy as np
# max([len(i) for i in test])

In [ ]:
# test = pd.DataFrame(pca.transform(test))
# test.to_csv('./test_600_pca.csv')

# print(sum(pca.explained_variance_ratio_))

In [ ]:
target = pd.read_csv('./origin_600_pca.csv')
conc = pd.DataFrame([list(reversed(list(target.iloc[i,:]))) for i in range(len(target))])

In [ ]:
conc = conc.rename(columns={i:f'pca_{i}' for i in range(600)})
target = target.rename(columns={f'{i}':f'pca_{i}' for i in range(600)}).drop('Unnamed: 0',axis=1)

In [ ]:
target = pd.concat([target,conc])
target

In [9]:
vector = pd.DataFrame(vector,columns=[f'mfcc_{i}' for i in range(64)])
train = pd.DataFrame(train_mel,columns=[f'mel_{i}' for i in range(64)])
# train = train.rename(columns={f'{i}':f'mel_{i}' for i in range(128)})
train = pd.concat([train,vector],axis=1)
# train = pd.concat([train,tp],axis=1)
# del(tp)
train

,mel_0,mel_1,mel_2,mel_3,mel_4,mel_5,mel_6,mel_7,mel_8,mel_9,...,mfcc_54,mfcc_55,mfcc_56,mfcc_57,mfcc_58,mfcc_59,mfcc_60,mfcc_61,mfcc_62,mfcc_63
0,0.066779,0.092294,0.091007,0.116941,0.114948,0.047119,0.037005,0.045050,0.044752,0.021855,...,2.829365,2.442121,1.998708,1.402770,-0.094236,-0.041829,0.312695,0.695733,1.466756,0.969668
1,0.065149,0.065050,0.107233,0.123646,0.114991,0.061604,0.078864,0.038930,0.025847,0.036424,...,0.248797,2.043477,3.339698,3.270855,1.894282,0.698431,-0.590722,1.163717,0.133544,-0.628808
2,0.063477,0.073673,0.124614,0.146078,0.145183,0.163153,0.099334,0.128659,0.131680,0.215220,...,1.076275,1.072367,0.156450,0.628312,0.578373,0.755597,-0.668103,-1.502379,-0.518559,0.458713
3,0.054856,0.118854,0.109508,0.102888,0.123376,0.117094,0.163167,0.286029,0.296922,0.189185,...,-1.546709,0.944595,1.389473,2.566277,0.567485,-0.663395,0.149160,-1.530205,0.713372,-0.100580
4,0.056592,0.100303,0.163865,0.159117,0.221757,0.183503,0.211223,0.207461,0.158711,0.262129,...,2.906172,2.612946,0.658167,-0.804220,0.620701,1.512230,1.035760,0.305853,-0.400905,0.693600
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4996,0.061863,0.113238,0.093037,0.124667,0.120672,0.108512,0.111071,0.111648,0.077524,0.047587,...,-1.681391,-1.243550,-0.895665,-1.086830,0.505544,0.290336,-0.582048,-0.138738,-0.930851,-1.878463
4997,0.071962,0.093476,0.087518,0.099995,0.156054,0.251519,0.664855,0.508591,0.141399,0.073500,...,3.604691,1.926490,1.580529,1.010258,0.817413,0.841381,0.145495,0.760276,0.284756,1.317319
4998,0.078089,0.068359,0.131051,0.110353,0.145028,0.195545,0.088464,0.054596,0.100964,0.073463,...,0.431391,0.913676,1.849735,1.892341,1.709681,2.131955,-0.813240,-0.233484,0.330278,0.528793
4999,0.061680,0.089095,0.080302,0.098390,0.125978,0.058736,0.040418,0.075699,0.035021,0.036867,...,-0.081796,1.222345,1.881899,2.106781,1.572352,1.599251,-0.203526,0.769150,-0.163830,0.241779


In [10]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [48]:
def build_model():
    model = keras.Sequential([
        layers.Dense(256, activation='relu', input_shape=[len(train.keys())]),
        # layers.MaxPooling1D(pool_size=2,strides=1, padding='valid'),
        layers.Dense(512),
        layers.Dropout(0.1),
        layers.Dense(6,activation='softmax')
        ])

    optimizer = tf.keras.optimizers.Adam(0.005)

    model.compile(loss=tf.keras.losses.sparse_categorical_crossentropy,
                optimizer='adam',
                metrics=['accuracy'])
    return model
model = build_model()

In [49]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_16 (Dense)            (None, 256)               33024     
                                                                 
 dense_17 (Dense)            (None, 512)               131584    
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_18 (Dense)            (None, 6)                 3078      
                                                                 
Total params: 167,686
Trainable params: 167,686
Non-trainable params: 0
_________________________________________________________________


In [14]:
import sklearn
# skf = sklearn.model_selection.StratifiedKFold(n_splits=10)
# for i, (train_index, test_index) in enumerate(skf.split(train, train_df['label'])):
#     print(f"Fold {i}:")
#     print(f"  Train: index={train_index}")
#     print(f"  Test:  index={test_index}")

In [51]:
# class PrintDot(keras.callbacks.Callback):
#     def on_epoch_end(self, epoch, logs):
#         if epoch % 100 == 0: print('')
#         print('.', end='')

# EPOCHS = 1000

# history = model.fit(
#   train, train_df['label'],
#   epochs=EPOCHS, validation_split = 0.1, verbose=0,
#   callbacks=[PrintDot()])



skf = sklearn.model_selection.KFold(n_splits=5)
models = []
for i, (train_index, test_index) in enumerate(skf.split(train, train_df['label'])):
    model.fit(train.loc[train_index,:]
              , train_df.loc[train_index,'label']
              , epochs=50
              , validation_data=(train.loc[test_index,:],train_df.loc[test_index,'label'])) # validation_split=0.1,
    models.append(model)

Epoch 1/50
125/125 [==============================] - 1s 6ms/step - loss: 1.0901 - accuracy: 0.5605 - val_loss: 1.2498 - val_accuracy: 0.5130
Epoch 2/50
125/125 [==============================] - 1s 6ms/step - loss: 1.0435 - accuracy: 0.5857 - val_loss: 1.1464 - val_accuracy: 0.5550
Epoch 3/50
125/125 [==============================] - 1s 6ms/step - loss: 1.0224 - accuracy: 0.5932 - val_loss: 1.0994 - val_accuracy: 0.5790
Epoch 4/50
125/125 [==============================] - 1s 6ms/step - loss: 1.0018 - accuracy: 0.6060 - val_loss: 1.0659 - val_accuracy: 0.5850
Epoch 5/50
125/125 [==============================] - 1s 6ms/step - loss: 0.9878 - accuracy: 0.5987 - val_loss: 1.0489 - val_accuracy: 0.5810
Epoch 6/50
125/125 [==============================] - 1s 6ms/step - loss: 0.9558 - accuracy: 0.6263 - val_loss: 1.0118 - val_accuracy: 0.6000
Epoch 7/50
125/125 [==============================] - 1s 6ms/step - loss: 0.9567 - accuracy: 0.6248 - val_loss: 1.0452 - val_accuracy: 0.6040
Epoch 

126/126 [==============================] - 1s 6ms/step - loss: 0.5971 - accuracy: 0.7688 - val_loss: 0.7954 - val_accuracy: 0.7070
Epoch 16/50
126/126 [==============================] - 1s 7ms/step - loss: 0.6625 - accuracy: 0.7463 - val_loss: 0.8624 - val_accuracy: 0.6760
Epoch 17/50
126/126 [==============================] - 1s 6ms/step - loss: 0.6042 - accuracy: 0.7683 - val_loss: 0.7802 - val_accuracy: 0.7090
Epoch 18/50
126/126 [==============================] - 1s 7ms/step - loss: 0.6076 - accuracy: 0.7671 - val_loss: 0.9227 - val_accuracy: 0.6750
Epoch 19/50
126/126 [==============================] - 1s 6ms/step - loss: 0.5731 - accuracy: 0.7833 - val_loss: 1.0199 - val_accuracy: 0.6580
Epoch 20/50
126/126 [==============================] - 1s 7ms/step - loss: 0.5965 - accuracy: 0.7646 - val_loss: 0.8645 - val_accuracy: 0.6890
Epoch 21/50
126/126 [==============================] - 1s 6ms/step - loss: 0.5684 - accuracy: 0.7791 - val_loss: 0.9467 - val_accuracy: 0.6760
Epoch 22/50

126/126 [==============================] - 1s 6ms/step - loss: 0.5132 - accuracy: 0.7966 - val_loss: 0.8142 - val_accuracy: 0.7260
Epoch 30/50
126/126 [==============================] - 1s 6ms/step - loss: 0.4725 - accuracy: 0.8158 - val_loss: 0.8000 - val_accuracy: 0.7170
Epoch 31/50
126/126 [==============================] - 1s 7ms/step - loss: 0.4293 - accuracy: 0.8303 - val_loss: 0.8817 - val_accuracy: 0.6970
Epoch 32/50
126/126 [==============================] - 1s 7ms/step - loss: 0.4719 - accuracy: 0.8128 - val_loss: 1.0356 - val_accuracy: 0.6890
Epoch 33/50
126/126 [==============================] - 1s 7ms/step - loss: 0.4857 - accuracy: 0.8103 - val_loss: 0.8894 - val_accuracy: 0.7240
Epoch 34/50
126/126 [==============================] - 1s 6ms/step - loss: 0.5145 - accuracy: 0.8063 - val_loss: 0.9601 - val_accuracy: 0.6970
Epoch 35/50
126/126 [==============================] - 1s 6ms/step - loss: 0.4315 - accuracy: 0.8338 - val_loss: 0.9760 - val_accuracy: 0.6910
Epoch 36/50

In [ ]:
# from pycaret import classification
# classification.setup(data=train,target='label')

ImportError: cannot import name 'classification' from 'pycaret' (/Users/junho/miniforge3/envs/ml_dl/lib/python3.8/site-packages/pycaret/__init__.py)

In [29]:
test = pd.DataFrame(test_mel,columns=[f'mel_{i}' for i in range(64)])
vector = pd.DataFrame(test_mfcc,columns=[f'mfcc_{i}' for i in range(64)])
test = pd.concat([test,vector],axis=1)

In [30]:
models

In [35]:
preds = [i.predict(test) for i in models]
preds = [np.argmax(i) for i in np.array(preds).mean(axis=0)]
len(preds)

59/59 [==============================] - 0s 3ms/step


In [36]:
submission = pd.read_csv('./sample_submission.csv')
submission['label'] = preds # 0,2,3
submission.to_csv('./baseline_submission.csv', index=False)